In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from collections import Counter

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow import keras as K

from categorization import data
from categorization.featurization import NameAndReviewTextFeaturizer
from categorization.evaluation import experiment, model_experiment, evaluate_model
from categorization.model import DenseTextualModel, RnnTextualModel

## Build the Train and Validation Datasets

In [4]:
businesses = data.load_business_df()
len(businesses)

158525

In [5]:
pen_businesses = businesses[businesses.state == 'PA']

In [6]:
pen_businesses.head()

,business_id,business_name,review_count,stars,state,city,categories
20,1RHY4K3BD22FK7Cfftn8Mg,Marathon Diner,35,4.0,PA,Pittsburgh,"[Sandwiches, Salad, Restaurants, Burgers, Comf..."
43,qWWBVE5T_zMEF7UJ4iTfNw,"DJ Yonish, Inc.",3,2.5,PA,Bethel Park,"[Home Services, Heating & Air Conditioning/HVAC]"
51,dQj5DLZjeDK3KFysh1SYOQ,Apteka,242,4.5,PA,Pittsburgh,"[Nightlife, Bars, Polish, Modern European, Res..."
58,v-scZMU6jhnmV955RSzGJw,No. 1 Sushi Sushi,106,4.5,PA,Pittsburgh,"[Japanese, Sushi Bars, Restaurants]"
61,KFbUQ-RR2UOV62Ep7WnXHw,Westwood Bar & Grill,5,3.0,PA,West Mifflin,"[American (Traditional), Restaurants]"


In [7]:
root_categories = data.CategoryTree().root_categories

In [8]:
examples, label_sets = data.load_examples(
    set(pen_businesses.business_id.unique()),
    min_reviews=1,
    accepted_categories=root_categories,
)

len(examples), len(label_sets)

(9411, 9411)

In [9]:
train_examples, validation_examples, train_label_sets, validation_label_sets = \
    train_test_split(examples, label_sets, test_size=.2)

len(train_examples), len(train_label_sets), len(validation_examples), len(validation_label_sets)

(7528, 7528, 1883, 1883)

## Models

In [10]:
labelizer = MultiLabelBinarizer()
labelizer.fit(train_label_sets)


NUM_CLASSES = len({label for labels in label_sets for label in labels})

In [16]:
vocab_size = 5000
input_length = 10000

featurizer = NameAndReviewTextFeaturizer(max_vocab_size=vocab_size, max_length=input_length)
featurizer.fit(train_examples)

In [17]:
train_features = featurizer.transform(train_examples)
validation_features = featurizer.transform(validation_examples)

In [18]:
dense_experiment = model_experiment(
    featurizer,
    DenseTextualModel(
        vocab_size=vocab_size, input_length=input_length,
        embedding_dimension=50, hidden_dimension=50, num_classes=NUM_CLASSES,
        learning_rate=0.01, epochs=200, batch_size=256
    ),
    labelizer,
    train_features,
    train_label_sets, validation_features, validation_label_sets, 
)

Epoch 1/200
30/30 [==============================] - 7s 249ms/step - loss: 0.3538 - accuracy: 0.1666 - val_loss: 0.2214 - val_accuracy: 0.1572
Epoch 2/200
30/30 [==============================] - 7s 238ms/step - loss: 0.2135 - accuracy: 0.1708 - val_loss: 0.2106 - val_accuracy: 0.1636
Epoch 3/200
30/30 [==============================] - 7s 237ms/step - loss: 0.2038 - accuracy: 0.1841 - val_loss: 0.2052 - val_accuracy: 0.1673
Epoch 4/200
30/30 [==============================] - 7s 236ms/step - loss: 0.1998 - accuracy: 0.1906 - val_loss: 0.2010 - val_accuracy: 0.1753
Epoch 5/200
30/30 [==============================] - 7s 236ms/step - loss: 0.1947 - accuracy: 0.2076 - val_loss: 0.1941 - val_accuracy: 0.2039
Epoch 6/200
30/30 [==============================] - 7s 234ms/step - loss: 0.1875 - accuracy: 0.2318 - val_loss: 0.1872 - val_accuracy: 0.2289
Epoch 7/200
30/30 [==============================] - 7s 236ms/step - loss: 0.1820 - accuracy: 0.2451 - val_loss: 0.1834 - val_accuracy: 0.1986

Epoch 58/200
30/30 [==============================] - 7s 235ms/step - loss: 0.0447 - accuracy: 0.7411 - val_loss: 0.1135 - val_accuracy: 0.7074
Epoch 59/200
30/30 [==============================] - 7s 235ms/step - loss: 0.0431 - accuracy: 0.7457 - val_loss: 0.1120 - val_accuracy: 0.7100
Epoch 60/200
30/30 [==============================] - 7s 235ms/step - loss: 0.0432 - accuracy: 0.7431 - val_loss: 0.1129 - val_accuracy: 0.7138
Epoch 61/200
30/30 [==============================] - 7s 233ms/step - loss: 0.0419 - accuracy: 0.7476 - val_loss: 0.1208 - val_accuracy: 0.6798
Epoch 62/200
30/30 [==============================] - 7s 237ms/step - loss: 0.0412 - accuracy: 0.7436 - val_loss: 0.1176 - val_accuracy: 0.7084
Epoch 63/200
30/30 [==============================] - 7s 233ms/step - loss: 0.0402 - accuracy: 0.7529 - val_loss: 0.1205 - val_accuracy: 0.6819
Epoch 64/200
30/30 [==============================] - 7s 230ms/step - loss: 0.0400 - accuracy: 0.7492 - val_loss: 0.1210 - val_accuracy:

Epoch 115/200
30/30 [==============================] - 7s 238ms/step - loss: 0.0106 - accuracy: 0.7836 - val_loss: 0.2083 - val_accuracy: 0.7031
Epoch 116/200
30/30 [==============================] - 7s 235ms/step - loss: 0.0118 - accuracy: 0.7819 - val_loss: 0.2122 - val_accuracy: 0.7106
Epoch 117/200
30/30 [==============================] - 7s 236ms/step - loss: 0.0115 - accuracy: 0.7841 - val_loss: 0.2119 - val_accuracy: 0.7207
Epoch 118/200
30/30 [==============================] - 7s 234ms/step - loss: 0.0098 - accuracy: 0.7836 - val_loss: 0.2107 - val_accuracy: 0.7180
Epoch 119/200
30/30 [==============================] - 7s 237ms/step - loss: 0.0097 - accuracy: 0.7843 - val_loss: 0.2148 - val_accuracy: 0.7132
Epoch 120/200
30/30 [==============================] - 7s 237ms/step - loss: 0.0094 - accuracy: 0.7853 - val_loss: 0.2199 - val_accuracy: 0.7127
Epoch 121/200
30/30 [==============================] - 7s 237ms/step - loss: 0.0109 - accuracy: 0.7849 - val_loss: 0.2242 - val_ac

30/30 [==============================] - 7s 235ms/step - loss: 0.0041 - accuracy: 0.7863 - val_loss: 0.3079 - val_accuracy: 0.7053
Epoch 172/200
30/30 [==============================] - 7s 234ms/step - loss: 0.0028 - accuracy: 0.7843 - val_loss: 0.3094 - val_accuracy: 0.7069
Epoch 173/200
30/30 [==============================] - 7s 233ms/step - loss: 0.0024 - accuracy: 0.7863 - val_loss: 0.3128 - val_accuracy: 0.7058
Epoch 174/200
30/30 [==============================] - 7s 242ms/step - loss: 0.0022 - accuracy: 0.7844 - val_loss: 0.3118 - val_accuracy: 0.7095
Epoch 175/200
30/30 [==============================] - 7s 235ms/step - loss: 0.0025 - accuracy: 0.7829 - val_loss: 0.3152 - val_accuracy: 0.7143
Epoch 176/200
30/30 [==============================] - 7s 236ms/step - loss: 0.0023 - accuracy: 0.7836 - val_loss: 0.3188 - val_accuracy: 0.7138
Epoch 177/200
30/30 [==============================] - 7s 235ms/step - loss: 0.0023 - accuracy: 0.7855 - val_loss: 0.3184 - val_accuracy: 0.7132

In [19]:
dense_experiment_2 = model_experiment(
    featurizer,
    DenseTextualModel(
        vocab_size=vocab_size, input_length=input_length,
        embedding_dimension=100, hidden_dimension=100, num_classes=NUM_CLASSES,
        learning_rate=0.001, epochs=500, batch_size=512
    ),
    labelizer,
    train_features,
    train_label_sets, validation_features, validation_label_sets, 
)

Epoch 1/500
15/15 [==============================] - 13s 839ms/step - loss: 0.6745 - accuracy: 0.0461 - val_loss: 0.6471 - val_accuracy: 0.0430
Epoch 2/500
15/15 [==============================] - 13s 833ms/step - loss: 0.6074 - accuracy: 0.0401 - val_loss: 0.5439 - val_accuracy: 0.0212
Epoch 3/500
15/15 [==============================] - 12s 832ms/step - loss: 0.4665 - accuracy: 0.0247 - val_loss: 0.3669 - val_accuracy: 0.0212
Epoch 4/500
15/15 [==============================] - 13s 836ms/step - loss: 0.3021 - accuracy: 0.0287 - val_loss: 0.2501 - val_accuracy: 0.0542
Epoch 5/500
15/15 [==============================] - 13s 839ms/step - loss: 0.2354 - accuracy: 0.1562 - val_loss: 0.2281 - val_accuracy: 0.1572
Epoch 6/500
15/15 [==============================] - 12s 831ms/step - loss: 0.2218 - accuracy: 0.1707 - val_loss: 0.2205 - val_accuracy: 0.1572
Epoch 7/500
15/15 [==============================] - 13s 838ms/step - loss: 0.2164 - accuracy: 0.1707 - val_loss: 0.2170 - val_accuracy:

15/15 [==============================] - 13s 835ms/step - loss: 0.1815 - accuracy: 0.2492 - val_loss: 0.1846 - val_accuracy: 0.2321
Epoch 58/500
15/15 [==============================] - 12s 830ms/step - loss: 0.1811 - accuracy: 0.2472 - val_loss: 0.1843 - val_accuracy: 0.2278
Epoch 59/500
15/15 [==============================] - 13s 833ms/step - loss: 0.1806 - accuracy: 0.2454 - val_loss: 0.1838 - val_accuracy: 0.2331
Epoch 60/500
15/15 [==============================] - 13s 837ms/step - loss: 0.1801 - accuracy: 0.2508 - val_loss: 0.1834 - val_accuracy: 0.2284
Epoch 61/500
15/15 [==============================] - 13s 844ms/step - loss: 0.1797 - accuracy: 0.2519 - val_loss: 0.1830 - val_accuracy: 0.2262
Epoch 62/500
15/15 [==============================] - 13s 836ms/step - loss: 0.1792 - accuracy: 0.2499 - val_loss: 0.1824 - val_accuracy: 0.2337
Epoch 63/500
15/15 [==============================] - 13s 839ms/step - loss: 0.1787 - accuracy: 0.2508 - val_loss: 0.1820 - val_accuracy: 0.233

15/15 [==============================] - 13s 839ms/step - loss: 0.1549 - accuracy: 0.3378 - val_loss: 0.1598 - val_accuracy: 0.3181
Epoch 114/500
15/15 [==============================] - 13s 835ms/step - loss: 0.1543 - accuracy: 0.3440 - val_loss: 0.1592 - val_accuracy: 0.3240
Epoch 115/500
15/15 [==============================] - 13s 837ms/step - loss: 0.1536 - accuracy: 0.3484 - val_loss: 0.1584 - val_accuracy: 0.3245
Epoch 116/500
15/15 [==============================] - 13s 838ms/step - loss: 0.1528 - accuracy: 0.3545 - val_loss: 0.1578 - val_accuracy: 0.3293
Epoch 117/500
15/15 [==============================] - 13s 837ms/step - loss: 0.1522 - accuracy: 0.3540 - val_loss: 0.1572 - val_accuracy: 0.3293
Epoch 118/500
15/15 [==============================] - 13s 836ms/step - loss: 0.1514 - accuracy: 0.3654 - val_loss: 0.1563 - val_accuracy: 0.3532
Epoch 119/500
15/15 [==============================] - 12s 831ms/step - loss: 0.1507 - accuracy: 0.3711 - val_loss: 0.1558 - val_accuracy:

15/15 [==============================] - 13s 838ms/step - loss: 0.1223 - accuracy: 0.5495 - val_loss: 0.1305 - val_accuracy: 0.5539
Epoch 170/500
15/15 [==============================] - 13s 839ms/step - loss: 0.1217 - accuracy: 0.5506 - val_loss: 0.1297 - val_accuracy: 0.5597
Epoch 171/500
15/15 [==============================] - 13s 834ms/step - loss: 0.1211 - accuracy: 0.5530 - val_loss: 0.1295 - val_accuracy: 0.5539
Epoch 172/500
15/15 [==============================] - 13s 834ms/step - loss: 0.1208 - accuracy: 0.5594 - val_loss: 0.1292 - val_accuracy: 0.5491
Epoch 173/500
15/15 [==============================] - 13s 839ms/step - loss: 0.1204 - accuracy: 0.5553 - val_loss: 0.1290 - val_accuracy: 0.5651
Epoch 174/500
15/15 [==============================] - 13s 838ms/step - loss: 0.1199 - accuracy: 0.5579 - val_loss: 0.1285 - val_accuracy: 0.5571
Epoch 175/500
15/15 [==============================] - 13s 837ms/step - loss: 0.1195 - accuracy: 0.5634 - val_loss: 0.1282 - val_accuracy:

15/15 [==============================] - 13s 837ms/step - loss: 0.1012 - accuracy: 0.6168 - val_loss: 0.1153 - val_accuracy: 0.6192
Epoch 226/500
15/15 [==============================] - 13s 836ms/step - loss: 0.1010 - accuracy: 0.6161 - val_loss: 0.1153 - val_accuracy: 0.6150
Epoch 227/500
15/15 [==============================] - 12s 829ms/step - loss: 0.1007 - accuracy: 0.6172 - val_loss: 0.1149 - val_accuracy: 0.6224
Epoch 228/500
15/15 [==============================] - 13s 848ms/step - loss: 0.1003 - accuracy: 0.6178 - val_loss: 0.1149 - val_accuracy: 0.6192
Epoch 229/500
15/15 [==============================] - 12s 829ms/step - loss: 0.1000 - accuracy: 0.6192 - val_loss: 0.1147 - val_accuracy: 0.6229
Epoch 230/500
15/15 [==============================] - 12s 830ms/step - loss: 0.0997 - accuracy: 0.6226 - val_loss: 0.1143 - val_accuracy: 0.6229
Epoch 231/500
15/15 [==============================] - 12s 831ms/step - loss: 0.0994 - accuracy: 0.6215 - val_loss: 0.1143 - val_accuracy:

15/15 [==============================] - 12s 831ms/step - loss: 0.0855 - accuracy: 0.6509 - val_loss: 0.1070 - val_accuracy: 0.6468
Epoch 282/500
15/15 [==============================] - 13s 839ms/step - loss: 0.0853 - accuracy: 0.6521 - val_loss: 0.1069 - val_accuracy: 0.6532
Epoch 283/500
15/15 [==============================] - 13s 834ms/step - loss: 0.0852 - accuracy: 0.6536 - val_loss: 0.1068 - val_accuracy: 0.6458
Epoch 284/500
15/15 [==============================] - 12s 830ms/step - loss: 0.0848 - accuracy: 0.6526 - val_loss: 0.1067 - val_accuracy: 0.6516
Epoch 285/500
15/15 [==============================] - 12s 829ms/step - loss: 0.0845 - accuracy: 0.6546 - val_loss: 0.1070 - val_accuracy: 0.6527
Epoch 286/500
15/15 [==============================] - 13s 840ms/step - loss: 0.0843 - accuracy: 0.6537 - val_loss: 0.1069 - val_accuracy: 0.6479
Epoch 287/500
15/15 [==============================] - 13s 835ms/step - loss: 0.0841 - accuracy: 0.6569 - val_loss: 0.1064 - val_accuracy:

15/15 [==============================] - 12s 829ms/step - loss: 0.0727 - accuracy: 0.6783 - val_loss: 0.1036 - val_accuracy: 0.6644
Epoch 338/500
15/15 [==============================] - 13s 837ms/step - loss: 0.0723 - accuracy: 0.6792 - val_loss: 0.1033 - val_accuracy: 0.6654
Epoch 339/500
15/15 [==============================] - 13s 840ms/step - loss: 0.0720 - accuracy: 0.6800 - val_loss: 0.1033 - val_accuracy: 0.6622
Epoch 340/500
15/15 [==============================] - 13s 834ms/step - loss: 0.0718 - accuracy: 0.6799 - val_loss: 0.1032 - val_accuracy: 0.6713
Epoch 341/500
15/15 [==============================] - 13s 837ms/step - loss: 0.0716 - accuracy: 0.6812 - val_loss: 0.1033 - val_accuracy: 0.6665
Epoch 342/500
15/15 [==============================] - 12s 828ms/step - loss: 0.0714 - accuracy: 0.6788 - val_loss: 0.1034 - val_accuracy: 0.6713
Epoch 343/500
15/15 [==============================] - 13s 834ms/step - loss: 0.0713 - accuracy: 0.6816 - val_loss: 0.1031 - val_accuracy:

15/15 [==============================] - 12s 833ms/step - loss: 0.0618 - accuracy: 0.7007 - val_loss: 0.1035 - val_accuracy: 0.6814
Epoch 394/500
15/15 [==============================] - 13s 837ms/step - loss: 0.0616 - accuracy: 0.7015 - val_loss: 0.1032 - val_accuracy: 0.6776
Epoch 395/500
15/15 [==============================] - 13s 836ms/step - loss: 0.0615 - accuracy: 0.7007 - val_loss: 0.1039 - val_accuracy: 0.6830
Epoch 396/500
15/15 [==============================] - 13s 834ms/step - loss: 0.0614 - accuracy: 0.7027 - val_loss: 0.1032 - val_accuracy: 0.6808
Epoch 397/500
15/15 [==============================] - 12s 830ms/step - loss: 0.0611 - accuracy: 0.7024 - val_loss: 0.1039 - val_accuracy: 0.6814
Epoch 398/500
15/15 [==============================] - 13s 841ms/step - loss: 0.0610 - accuracy: 0.7032 - val_loss: 0.1034 - val_accuracy: 0.6782
Epoch 399/500
15/15 [==============================] - 13s 841ms/step - loss: 0.0609 - accuracy: 0.7024 - val_loss: 0.1034 - val_accuracy:

15/15 [==============================] - 13s 841ms/step - loss: 0.0522 - accuracy: 0.7221 - val_loss: 0.1058 - val_accuracy: 0.6952
Epoch 450/500
15/15 [==============================] - 12s 832ms/step - loss: 0.0520 - accuracy: 0.7214 - val_loss: 0.1060 - val_accuracy: 0.6962
Epoch 451/500
15/15 [==============================] - 13s 837ms/step - loss: 0.0518 - accuracy: 0.7200 - val_loss: 0.1059 - val_accuracy: 0.7010
Epoch 452/500
15/15 [==============================] - 13s 838ms/step - loss: 0.0517 - accuracy: 0.7250 - val_loss: 0.1061 - val_accuracy: 0.6968
Epoch 453/500
15/15 [==============================] - 13s 841ms/step - loss: 0.0515 - accuracy: 0.7224 - val_loss: 0.1075 - val_accuracy: 0.7026
Epoch 454/500
15/15 [==============================] - 13s 837ms/step - loss: 0.0513 - accuracy: 0.7253 - val_loss: 0.1064 - val_accuracy: 0.6999
Epoch 455/500
15/15 [==============================] - 13s 840ms/step - loss: 0.0510 - accuracy: 0.7240 - val_loss: 0.1064 - val_accuracy:

In [20]:
dense_experiment_2.train_metrics

,tp,fp,tn,fn,support,precision,recall,f1
Mass Media,0,0,7515,13,13,1.000000,0.000000,0.000000
Religious Organizations,1,0,7492,35,36,1.000000,0.027778,0.054054
Education,11,2,7414,101,112,0.846154,0.098214,0.176000
Professional Services,16,2,7373,137,153,0.888889,0.104575,0.187135
Public Services & Government,13,0,7447,68,81,1.000000,0.160494,0.276596
Local Flavor,15,3,7444,66,81,0.833333,0.185185,0.303030
Financial Services,22,0,7455,51,73,1.000000,0.301370,0.463158
Local Services,167,31,6991,339,506,0.843434,0.330040,0.474432
Event Planning & Services,254,18,7036,220,474,0.933824,0.535865,0.680965
Arts & Entertainment,160,16,7249,103,263,0.909091,0.608365,0.728929


In [21]:
dense_experiment_2.validation_metrics

,tp,fp,tn,fn,support,precision,recall,f1
Mass Media,0,0,1879,4,4,1.000000,0.000000,0.000000
Religious Organizations,0,0,1874,9,9,1.000000,0.000000,0.000000
Professional Services,1,3,1845,34,35,0.250000,0.028571,0.051282
Public Services & Government,1,1,1850,31,32,0.500000,0.031250,0.058824
Local Flavor,1,1,1866,15,16,0.500000,0.062500,0.111111
Education,3,1,1858,21,24,0.750000,0.125000,0.214286
Local Services,39,20,1724,100,139,0.661017,0.280576,0.393939
Financial Services,6,0,1860,17,23,1.000000,0.260870,0.413793
Event Planning & Services,41,16,1760,66,107,0.719298,0.383178,0.500000
Arts & Entertainment,29,6,1812,36,65,0.828571,0.446154,0.580000


In [ ]:
featurizer = NameAndReviewTextFeaturizer(max_vocab_size=vocab_size, max_length=input_length)
featurizer.fit(train_examples)
train_features = featurizer.transform(train_examples)
validation_features = featurizer.transform(validation_examples)

In [ ]:
dense_experiment = model_experiment(
    featurizer,
    DenseTextualModel(
        vocab_size=vocab_size, input_length=input_length,
        embedding_dimension=50, hidden_dimension=50, num_classes=NUM_CLASSES,
        learning_rate=0.01, epochs=200, batch_size=256
    ),
    labelizer,
    train_features,
    train_label_sets, validation_features, validation_label_sets, 
)

In [ ]:
train_labels = labelizer.transform(train_label_sets)
validation_labels = labelizer.transform(validation_label_sets)

In [24]:
from tensorflow import keras as K

In [26]:
K.layers.LSTM(30)

In [27]:
dense_experiment = model_experiment(
    featurizer,
    DenseTextualModel(
        vocab_size=vocab_size, input_length=input_length,
        embedding_dimension=50, hidden_dimension=50, num_classes=NUM_CLASSES,
        learning_rate=0.01, epochs=1, batch_size=256
    ),
    labelizer,
    train_features,
    train_label_sets, validation_features, validation_label_sets, 
)

30/30 [==============================] - 7s 240ms/step - loss: 0.3590 - accuracy: 0.1149 - val_loss: 0.2239 - val_accuracy: 0.1004
Train Macro:
precision    1.0
recall       0.0
f1           0.0
dtype: float64

Validation Macro:
precision    1.0
recall       0.0
f1           0.0
dtype: float64


In [23]:
rnn_experiment = model_experiment(
    featurizer,
    RnnTextualModel(
        vocab_size=vocab_size, input_length=input_length,
        embedding_dimension=100, rnn_dimension=100, num_classes=NUM_CLASSES,
        learning_rate=0.001, epochs=200, batch_size=512
    ),
    labelizer,
    train_features,
    train_label_sets, validation_features, validation_label_sets, 
)

Epoch 1/200


UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_5/lstm_1/StatefulPartitionedCall]]
	 [[gradient_tape/sequential_5/embedding_5/embedding_lookup/Reshape/_38]] [Op:__inference_train_function_93121]

Function call stack:
train_function -> train_function -> train_function


In [22]:
%%time


rnn_experiment = model_experiment(
    featurizer,
    RnnTextualModel(
        vocab_size=vocab_size, input_length=input_length,
        embedding_dimension=50, rnn_dimension=30, num_classes=NUM_CLASSES,
        learning_rate=0.01, epochs=1, batch_size=256
    ),
    labelizer,
    train_features,
    train_label_sets, validation_features, validation_label_sets, 
)

UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_4/lstm/StatefulPartitionedCall]]
	 [[gradient_tape/sequential_4/embedding_4/embedding_lookup/Reshape/_38]] [Op:__inference_train_function_89866]

Function call stack:
train_function -> train_function -> train_function
